In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import shutil
import os
from google.colab import drive
drive.mount('/content/drive')
S = np.load('/content/drive/My Drive/Toric Code 2D/3: FRRBM on 3x3/Data/S20_30.npy')
SS = np.array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
               [-1, +1, -1, -1, -1, -1, -1, +1, -1, -1, -1, -1, -1, +1, -1, -1, -1, -1],
               [-1, -1, -1, -1, -1, -1, -1, -1, -1, +1, +1, +1, -1, -1, -1, -1, -1, -1],
               [-1, +1, -1, -1, -1, -1, -1, +1, -1, +1, +1, +1, -1, +1, -1, -1, -1, -1]])

Variables = np.load('/content/drive/My Drive/Toric Code 2D/3: FRRBM on 3x3/Data/Variables10_10k.npy')

# pr is the shortcut for cosh function.
def pr(b, w, s1, s2, s3, s4):
    result = torch.cosh((b[0]+1j*b[1])+1j*((w[0]+1j*w[4])*s1+(w[1]+1j*w[5])*s2+(w[2]+1j*w[6])*s3+(w[3]+1j*w[7])*s4))
    return result

def prr(b, w, s1, s2, s3, s4):
    result = torch.cosh(b[0] + 1j * (w[0] * s1 + w[1] * s2 + w[2] * s3 + w[3] * s4))
    return result

# ph is the phase function.
def ph(bv, wv, bf, wf, s):
    result = (pr(bv, wv, s[0], s[3], s[2], s[15]) * pr(bv, wv, s[1], s[4], s[0], s[16])
              * pr(bv, wv, s[2], s[5], s[1], s[17]) * pr(bv, wv, s[6], s[9], s[8], s[3])
              * pr(bv, wv, s[7], s[10], s[6], s[4]) * pr(bv, wv, s[8], s[11], s[7], s[5])
              * pr(bv, wv, s[12], s[15], s[14], s[9]) * pr(bv, wv, s[13], s[16], s[12], s[10])
              * pr(bv, wv, s[14], s[17], s[13], s[11]) * prr(bf, wf, s[4], s[6], s[3], s[0])
              * prr(bf, wf, s[5], s[7], s[4], s[1]) * prr(bf, wf, s[3], s[8], s[5], s[2])
              * prr(bf, wf, s[10], s[12], s[9], s[6]) * prr(bf, wf, s[11], s[13], s[10], s[7])
              * prr(bf, wf, s[9], s[14], s[11], s[8]) * prr(bf, wf, s[16], s[0], s[15], s[12])
              * prr(bf, wf, s[17], s[1], s[16], s[13]) * prr(bf, wf, s[15], s[2], s[17], s[14]))
    return result

# nor is the normalize function.
def nor(bv, wv, bf, wf):
    result = 0
    for i in range(len(S)):
      result += abs(ph(bv, wv, bf, wf, S[i]))
    return result


# The 9 vertex operators flip qubits.
A1 = np.diag([-1, +1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, +1])
A2 = np.diag([-1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1])
A3 = np.diag([+1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1])
A4 = np.diag([+1, +1, +1, -1, +1, +1, -1, +1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1])
A5 = np.diag([+1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1])
A6 = np.diag([+1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1])
A7 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, +1, -1, +1, -1, -1, +1, +1])
A8 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1])
A9 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1])
S1 = np.dot(S, A1)
S2 = np.dot(S, A2)
S3 = np.dot(S, A3)
S4 = np.dot(S, A4)
S5 = np.dot(S, A5)
S6 = np.dot(S, A6)
S7 = np.dot(S, A7)
S8 = np.dot(S, A8)
S9 = np.dot(S, A9)

# lam is the multiplier to amplify the choice of ground state
def criterion(bv, wv, bf, wf):
    V = torch.empty((18*N,), dtype=torch.complex64)
    for k in range(N):
        sub = ph(bv, wv, bf, wf, S[k])
        V[k] = ph(bv, wv, bf, wf, S1[k]) - sub
        V[1*N+k] = ph(bv, wv, bf, wf, S2[k]) - sub
        V[2*N+k] = ph(bv, wv, bf, wf, S3[k]) - sub
        V[3*N+k] = ph(bv, wv, bf, wf, S4[k]) - sub
        V[4*N+k] = ph(bv, wv, bf, wf, S5[k]) - sub
        V[5*N+k] = ph(bv, wv, bf, wf, S6[k]) - sub
        V[6*N+k] = ph(bv, wv, bf, wf, S7[k]) - sub
        V[7*N+k] = ph(bv, wv, bf, wf, S8[k]) - sub
        V[8*N+k] = ph(bv, wv, bf, wf, S9[k]) - sub
        V[9*N+k] = (S[k][4]*S[k][6]*S[k][3]*S[k][0]-1) * sub
        V[10*N+k] = (S[k][5]*S[k][7]*S[k][4]*S[k][1]-1) * sub
        V[11*N+k] = (S[k][3]*S[k][8]*S[k][5]*S[k][2]-1) * sub
        V[12*N+k] = (S[k][10]*S[k][12]*S[k][9]*S[k][6]-1) * sub
        V[13*N+k] = (S[k][11]*S[k][13]*S[k][10]*S[k][7]-1) * sub
        V[14*N+k] = (S[k][9]*S[k][14]*S[k][11]*S[k][8]-1) * sub
        V[15*N+k] = (S[k][16]*S[k][0]*S[k][15]*S[k][12]-1) * sub
        V[16*N+k] = (S[k][17]*S[k][1]*S[k][16]*S[k][13]-1) * sub
        V[17*N+k] = (S[k][15]*S[k][2]*S[k][17]*S[k][14]-1) * sub
    v = torch.norm(V)/nor(bv, wv, bf, wf)
    return v

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_time = time.time()
(ss, tt, lam, N) = (0.02, 50, 1.0, len(S))
BF = torch.tensor([0.0])
WF = torch.tensor([np.pi/4, np.pi/4, np.pi/4, np.pi/4])
All = []
Select = []
for k in range(0, 100):
    IN = Variables[k]
    BV = torch.tensor([IN[0], IN[1]], requires_grad=True)
    WV = torch.tensor([IN[2], IN[3], IN[4], IN[5], IN[6], IN[7], IN[8], IN[9]], requires_grad=True)
    optimizer = torch.optim.Adam([BV, WV], lr=ss)
    for h in range(tt):
        cost = criterion(BV, WV, BF, WF)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        All.append(IN.tolist())
    print("%d: (%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f)->(%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f), cost=%.2e" % (k, IN[0].tolist(), IN[1].tolist(), IN[2].tolist(), IN[3].tolist(),IN[4].tolist(), IN[5].tolist(), IN[6].tolist(), IN[7].tolist(),IN[8].tolist(), IN[9].tolist(), BV[0].tolist(), BV[1].tolist(), WV[0].tolist(), WV[1].tolist(), WV[2].tolist(), WV[3].tolist(), WV[4].tolist(), WV[5].tolist(), WV[6].tolist(), WV[7].tolist(), cost))
    if cost < 0.01:
        Select.append(k)
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")
print(f"Select:{Select}")


0: (-0.52,1.39,-3.15,-1.25,-2.23,-2.57,-1.98,-0.97,-0.65,0.24)->(0.09,1.71,-2.96,-0.76,-3.04,-2.34,-1.30,-0.63,-0.45,0.56), cost=9.02e-01
1: (-0.51,1.17,-1.86,2.38,-2.98,1.07,-0.52,0.37,-2.27,-1.90)->(-0.62,1.36,-1.76,2.08,-3.70,1.61,-0.71,1.09,-1.81,-1.09), cost=1.18e+00
2: (1.89,2.95,-1.18,1.21,2.37,2.49,-2.61,-2.90,-2.08,2.38)->(2.20,3.19,-0.73,0.97,2.45,2.47,-2.09,-2.62,-2.05,2.66), cost=1.55e+00
3: (-2.53,-0.50,2.88,0.21,1.21,-1.16,1.17,2.11,-3.03,1.58)->(nan,nan,nan,nan,nan,nan,nan,nan,nan,nan), cost=nan
4: (3.08,1.56,-1.38,1.82,-2.50,-0.33,2.57,-1.30,-1.34,-2.33)->(nan,nan,nan,nan,nan,nan,nan,nan,nan,nan), cost=nan
5: (-3.03,1.13,-1.82,-1.48,-0.05,-2.81,0.47,-2.23,0.56,1.26)->(-2.65,0.66,-1.55,-1.91,0.03,-2.78,0.01,-1.61,-0.02,1.70), cost=3.84e-01
6: (-2.51,-0.54,1.22,-0.54,-2.84,0.23,1.03,0.09,2.80,0.55)->(-2.62,-0.68,1.05,-0.15,-2.69,0.11,0.90,-0.22,2.80,0.14), cost=9.44e-01
7: (2.54,-2.28,-2.27,1.94,-0.64,-2.11,2.69,-0.96,1.58,1.42)->(nan,nan,nan,nan,nan,nan,nan,nan,nan,nan),